In [1]:
import random
import motor.motor_asyncio
from dotenv import dotenv_values
from openimagingdatamodel.ontology_tools.radlex_concept import RadLexConcept
from openimagingdatamodel.ontology_tools.snomedct_concept import SnomedCTConcept

## Setup


In [2]:
config = dotenv_values(".env")
client = motor.motor_asyncio.AsyncIOMotorClient(config["ATLAS_DSN"])
# Ping the client to confirm
print(await client.server_info())  #

{'version': '7.0.11', 'gitVersion': 'f451220f0df2b9dfe073f1521837f8ec5c208a8c', 'modules': ['enterprise'], 'allocator': 'tcmalloc', 'javascriptEngine': 'mozjs', 'sysInfo': 'deprecated', 'versionArray': [7, 0, 11, 0], 'bits': 64, 'debug': False, 'maxBsonObjectSize': 16777216, 'storageEngines': ['devnull', 'inMemory', 'queryable_wt', 'wiredTiger'], 'ok': 1.0, '$clusterTime': {'clusterTime': Timestamp(1717160847, 14), 'signature': {'hash': b'\x9c*\x82\x0c\x82\x0bD\x08\xcf\xd9\x17w\xeb\xf3\xb1\x88f.v!', 'keyId': 7327016299177967635}}, 'operationTime': Timestamp(1717160847, 14)}


In [3]:
db = client["ontologies"]
radlex_collection = db["radlex"]
RADLEX_COUNT = await radlex_collection.count_documents({})
print(f"RADLEX_COUNT: {RADLEX_COUNT}")
snomedct_collection = db["snomedct"]
SNOMEDCT_COUNT = await snomedct_collection.count_documents({})
print(f"SNOMEDCT_COUNT: {SNOMEDCT_COUNT}")

RADLEX_COUNT: 46761
SNOMEDCT_COUNT: 508540


## SnomedCT Doc


In [4]:
raw_doc = await snomedct_collection.find_one({}, skip=random.randint(0, SNOMEDCT_COUNT))
print(raw_doc)

{'_id': '440839009', 'conceptId': '440839009', 'effectiveDate': '2009-01-31', 'modules': ['SNOMED-CT-core'], 'languageCode': 'en', 'preferredTerm': 'Turicibacter sanguinis', 'terms': [], 'semanticTags': ['organism'], 'caseSignificance': 'sensitive', 'definitions': None}


In [5]:
doc = SnomedCTConcept(**raw_doc)

In [6]:
print(doc.model_dump_json(indent=2))

{
  "concept_id": "440839009",
  "effective_date": "2009-01-31",
  "modules": [
    "SNOMED-CT-core"
  ],
  "language_code": "en",
  "preferred_term": "Turicibacter sanguinis",
  "terms": [],
  "case_significance": "sensitive",
  "definitions": null
}


## RadLex Doc


In [7]:
raw_doc = await radlex_collection.find_one({}, skip=random.randint(0, RADLEX_COUNT))
print(raw_doc)

{'_id': 'RID20925', 'preferredLabel': 'right lateral dorsal nucleus', 'parent': 'RID6608', 'radlexProperties': {'fmaid': '73015', 'preferredNameGerman': 'rechter Nucleus dorsalis lateralis', 'talairach': '788'}}


In [8]:
doc = RadLexConcept(**raw_doc)
print(doc.model_dump_json(indent=2))

{
  "id": "RID20925",
  "preferred_label": "right lateral dorsal nucleus",
  "synonyms": null,
  "parent": "RID6608",
  "definition": null,
  "radlex_properties": {
    "fmaid": "73015",
    "preferredNameGerman": "rechter Nucleus dorsalis lateralis",
    "talairach": "788"
  }
}


## SNOMED CT Repo


In [7]:
from openimagingdatamodel.ontology_tools.snomedct_concept_repo import SnomedCTConceptRepo

In [8]:
snomed_repo = SnomedCTConceptRepo(snomedct_collection)
doc = await snomed_repo.get_concept("113907005")
print(doc.model_dump_json(indent=2))

{
  "concept_id": "113907005",
  "effective_date": "2003-07-31",
  "modules": [
    "SNOMED-CT-core"
  ],
  "language_code": "en",
  "preferred_term": "Peptostreptococcus harei",
  "terms": [],
  "case_significance": "sensitive",
  "definitions": null
}


In [9]:
heart_docs = await snomed_repo.get_search_results("heart", 20)

In [10]:
for doc in heart_docs:
    print(doc.model_dump_json(indent=2))

{
  "concept_id": "85830006",
  "preferred_term": "Repair of heart valve",
  "semantic_tags": [
    "procedure"
  ],
  "score": 8.759529113769531
}
{
  "concept_id": "716661006",
  "preferred_term": "Childhood neoplasm of heart",
  "semantic_tags": [
    "disorder"
  ],
  "score": 8.428897857666016
}
{
  "concept_id": "715403006",
  "preferred_term": "Adult heart tumor",
  "semantic_tags": [
    "disorder"
  ],
  "score": 8.357369422912598
}
{
  "concept_id": "427565006",
  "preferred_term": "Absent fetal heart rate variability",
  "semantic_tags": [
    "finding"
  ],
  "score": 8.307586669921875
}
{
  "concept_id": "232971009",
  "preferred_term": "Heart and heart-lung transplant",
  "semantic_tags": [
    "procedure"
  ],
  "score": 7.662195205688477
}
{
  "concept_id": "387842002",
  "preferred_term": "Neoplasm of heart",
  "semantic_tags": [
    "disorder"
  ],
  "score": 7.362870216369629
}
{
  "concept_id": "32413006",
  "preferred_term": "Transplantation of heart",
  "semantic_